In [14]:
import grequests
import aiohttp
import asyncio
import requests
import time

token = 'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJlbWFpbCI6InBhdHRoYXJhcG9ybi50MTlAZ21haWwuY29tIiwidXJsYmFjayI6Ind3dy5hbGx0aWNrZXQuY29tIiwicGF5bWVudENoYW5uZWwiOiJDMDciLCJ0aWNrZXRUeXBlIjoiMDEiLCJsYW5nIjoiRSIsImRhdGEiOiI0ODQxMzEwOTIzNzc2MjgzYzdkZDY3YWIwMjc5NWIzOGVhNmYzNWU0MjFmYzI3ZWUwYWVhODJlMzY5Y2ZkZDc0NzNmOGM4YjU0Yzg3OTY5Y2I4Y2JhZTcyOGVhNDdjYmViNGZkZjI0M2JhNGQxMWE1YmMwZWFlYTQwYmFkNDcwYzZiNzJlZWMxZTBhY2Y2Mjc2OWMzZjc5NTI1YWEyZGFlYTIxMzFkNjczYmI0N2EyMDA0ODRjN2Y1MmI2MjlhMzI5YzQ2OGFmN2Y3NTNhOGIxY2Q4NDBkNDkwNjRkN2VhZjUyYmY5MmUzYWI1Mzg0ZmVhYjVhOTM5MzE5ZTliOWVmOWYwN2Q1YTAzNWU4ZjRjNzdlZWM1MWM2ZjUyNDQ2YjU4NWU0YWQ3NjU5YjhhMjllY2IxZjQxM2I2NTNiMTQ1MGE0OTgyODA0MjAzNjA4ODUyN2VmZjIyYmVkM2RhMTgxY2I1MDQ4MDU2MTIwNGM0OTE0YzY1Y2VhYjc3Y2Y2ZTA4NWU5Y2Q1ZGQ0YjVmY2ViNWY3MGFjYTRkZmM2NjIwZTFjYWE1YzI2NTQ3OGM4NDg0Y2U2MDUyODc3ODcxYzA1ZTgwMGY3YjgyYWI0NjAyNDlhMjVlMWQwNGU0Mjk3ZWM5M2U2YmFmNDVmOTZjNzk3YzNmYTE5ZTYzZDY4OWQzMTdlZTljOTVkMWU2Njc1MzU3MmY3ODJlYmI0OGMwYTY0N2E2NDVkN2I1OGJmMDliZmFkMmM1MjdkMjYzODYzOWZiNWNiZWY0NzM3ZmI4OGQ1NDU5MjAwOWJlNDgzZDE5ZDZkMjY1NDNkNDgyYjRlMDMzZGRlZTlmNWIwMGI4ZWJiZTJjOTk3NWM0ZTdiM2NkZmU0YmE5YTQ5YzkwODE5YTFhZWMxYTI1MDY0OTE2NTQ3Yjg3NjU2ZmEwNDY4YWZmMjM5NDQzMTNjMzBiYTI2NTUxYTllZWIwNmYxMTE5OWVkOTczN2I2Y2EyY2VhM2EzMDdlYWQ5ZTYzNDdlYmMwNDkiLCJ0aW1lU3RhbXAiOjAuNzYwOTM4OTY2MDA2NDkyNSwiaWF0IjoxNzAxNjE4NjMzLCJleHAiOjE3MDE2Mjk0MzMsImlzcyI6ImNzYXRrMTgifQ.fE2L6y4xUo0JEwB62x0Gj3jahzTuIFHZzW9enDBx9QI'
event = 'SEVENTEENFOLLOWTOBKK'

RETRY_SEAT_COUNT = 3

baseData = {
    'url': 'https://www.allticket.com/',
    'headers': {
        'authorization': token,
        'Content-Type': 'application/json'
    }
}

In [11]:
class PerformManager:
    id: int
    round_id: int
    token: str
    event: str
    seat_needed: int
    zones: list
    booked_seat: list

    def __init__(self, token, event, seat_needed = 3) -> None:
        self.token = token
        self.event = event
        self.seat_needed = seat_needed
        self.id = None
        self.round_id = None
        self.zone_greqs = []
        self.seat_greqs = []
        self.zones = self.get_zones()
        self.seats = self.get_seats()
        self.booked_seat = self.book_seats()
    
    def get_id(self):
        req = baseData.copy()
        req['url'] = 'https://api.allticket.com/content/get-home'
        res = requests.post(**req).json()
        items = res['data']['banner_']['items']
        found_con = next(filter(lambda x: x['performUri'] == self.event, items), None)
        if not found_con:
            raise Exception('NOT FOUND CONCERT THIS NAME')

        self.id = found_con['id']

    def get_round_id(self):
        if not self.id:
            self.get_id()        

        req = baseData.copy()
        req['url'] = 'https://api.allticket.com/booking/get-round'
        req['json'] = { 'performId': self.id }
        res = requests.post(**req).json()
        self.round_id = res['data']['event_info']['list_round'][0]['roundId']
        
    def get_zones(self):
        if not self.round_id:
            self.get_round_id()
        
        req = baseData.copy()
        req['url'] = 'https://api.allticket.com/booking/seat-available'
        req['json'] = { 'performId': self.id, 'roundId': self.round_id }
        res = requests.post(**req).json()
        
        zones = []
        for zone in res['data']['seat_available']:
            if zone['amount'] <= self.seat_needed:
                continue
            
            zone_manager = ZoneManager(self, zone['id'])
            zones.append(zone_manager)
            self.zone_greqs.append(grequests.post(**zone_manager.req))

        return zones
    
    def get_seats(self):
        seat_needed = self.seat_needed

        for i, res in grequests.imap_enumerated(self.zone_greqs):
            zone = self.zones[i]
            data = res.json()['data']
        
            if not data:
                continue

            zone.zone_type = data['zone_type']
            seats = []

            for seat_data in data['seats_available']:
                screen_label = seat_data['screenLabel']
                
                seat_ids = []
                count = 0
                for seat in seat_data['seat']:
                    status = seat['status']
                    
                    if status != "A":
                        continue

                    seat_ids.append(f"{seat['rowName']}_{seat['seatNo']}")
    
                    count += 1
                    
                    if count >= seat_needed:
                        count = 0
                        
                        seat_manager = SeatManager(zone, screen_label, seat_ids)
                        seats.append(seat_manager)
                        self.seat_greqs.append(seat_manager.req)
                        seat_ids = []

            return seats
    
    def book_seats(self):
        booked_seats = []
        payment_reqs = []
        
        for i, res in grequests.imap_enumerated(self.seat_greqs):
            seat = self.seats[i]
            data = res.json()['data']

            seat.booking_id = data['uuid']
            seat.booking_wait_time = data['waitTime']
            seat.booking_time_out = data['timeOut']
            seat.booking_retry = data['retry']
            
            if len(payment_reqs) >= self.seat_needed:
                break

            req = seat.get_payment_req()
            if (req):
                payment_reqs.append(req)
            
            

class ZoneManager:
    id: str
    zone_type: str
    perform: PerformManager
    seat_reqs = []
    seats: list

    def __init__(self, perform, zone_id) -> None:
        self.perform = perform
        self.id = zone_id
        self.req = self.get_req()
        self.seat_greqs = []
        # self.seats = self.get_seats()
        
    def done(self):
        self.perform.seat_needed <= 0
        
    def get_req(self):
        if self.done():
            return []
        
        req = baseData.copy()
        perform = self.perform

        req['url'] = 'https://api.allticket.com/booking/get-seat'
        req['json'] = { 'performId': perform.id, 'roundId': perform.round_id, 'zoneId': self.id }
        
        return req

    def get_seats(self):
        if self.done():
            return []
        
        perform = self.perform

        seat_needed = perform.seat_needed
        res = requests.post(**self.req).json()
        data = res['data']
        
        if not data:
            return None

        self.zone_type = data['zone_type']
        seats = []

        for seat_data in data['seats_available']:
            screen_label = seat_data['screenLabel']
            
            seat_ids = []
            count = 0
            for seat in seat_data['seat']:
                status = seat['status']
                
                if status != "A":
                    continue

                seat_ids.append(f"{seat['rowName']}_{seat['seatNo']}")
  
                count += 1
                
                if count >= seat_needed:
                    count = 0
                    
                    if self.done():
                        return seats
                    
                    seat_manager = SeatManager(self, screen_label, seat_ids)
                    if seat_manager.booked:
                        seats.append(seat_manager)

                    seat_ids = []

        return seats
    
class SeatManager:
    screen_label: str
    ids: list
    zone: ZoneManager
    booking_id = None
    booking_wait_time = 5
    booking_time_out = 20
    booking_retry = 9
    reserve_id = None
    booked = False

    def __init__(self, zone, screen_label, seat_ids) -> None:
        self.zone = zone
        self.screen_label = screen_label
        self.ids = seat_ids
        self.req = self.get_req()

    def done(self):
        return self.zone.perform.seat_needed <= 0
    
    def get_req(self):
        zone = self.zone
        perform = zone.perform

        req = baseData.copy()
        req['url'] = 'https://api.allticket.com/booking/handler-reserve'
        req['json'] = {
            "performId": perform.id,
            "roundId": perform.round_id,
            "zoneId": zone.id,
            "screenLabel": self.screen_label,
            "seatTo": {
                "seatType": zone.zone_type,
                "seats": self.ids
            },
            "shirtTo": []
        }
        
        return req

    def book(self):
        res = requests.post(**self.req).json()
        data = res['data']
        self.booking_id = data['uuid']
        self.booking_wait_time = data['waitTime']
        self.booking_time_out = data['timeOut']
        self.booking_retry = data['retry']

    def check_booking(self, req=None, count=0, time_count=0, force_retry_time=None, reset_counter=0):
        if self.done():
            return False
        
        if not self.booking_id:
            self.book()
            
        if force_retry_time is None:
            force_retry_time = self.booking_wait_time

        start_time = time.time()

        if req is None:
            req = baseData.copy()
            req['url'] = 'https://api.allticket.com/booking/check-booking'
            req['json'] = { "uuid": self.booking_id }

        time.sleep(self.booking_wait_time)
            
        res = requests.post(**req).json()
        data = res['data']
        success = data['success'] == True

        if success:
            self.reserve_id = data['reserveId']
            return True
        
        # retry Once
        if count == 0:
            self.check_booking(req, count+1, time.time() - start_time, force_retry_time=0.5)
            
        if reset_counter > RETRY_SEAT_COUNT:
            return False

        if count >= self.booking_retry or time_count >= self.booking_time_out:
            self.booking_id = None
            return self.check_booking(req, reset_counter=reset_counter+1)

        return self.check_booking(req, count+1, time.time() - start_time)
    
    def get_payment_req(self, req=None, count=0, time_count=0, force_retry_time=None, reset_counter=0):
        if self.done():
            return None
        
        if force_retry_time is None:
            force_retry_time = self.booking_wait_time

        start_time = time.time()

        if req is None:
            req = baseData.copy()
            req['url'] = 'https://api.allticket.com/booking/check-booking'
            req['json'] = { "uuid": self.booking_id }

        time.sleep(self.booking_wait_time)
        
        async with aiohttp.ClientSession() as session:
            async with session.get('http://python.org') as response:
                res = response.json()
                data = res['data']
                success = data['success'] == True

                if success:
                    self.reserve_id = data['reserveId']
                    return True
                
                # retry Once
                if count == 0:
                    self.check_booking(req, count+1, time.time() - start_time, force_retry_time=0.5)
                    
                if reset_counter > RETRY_SEAT_COUNT:
                    return False

                if count >= self.booking_retry or time_count >= self.booking_time_out:
                    self.booking_id = None
                    return self.check_booking(req, reset_counter=reset_counter+1)

                return self.check_booking(req, count+1, time.time() - start_time)
            
            
    def discount_normal(self):
        req = baseData.copy()
        req['url'] = 'https://api.allticket.com/booking/discount-normal'
        req['json'] = { 'reserveId': self.reserve_id }
        return requests.post(**req).json()
    
    def get_insurance_product(self):
        req = baseData.copy()
        req['url'] = f"https://api.allticket.com/booking/get-insure-product?reserveKey={self.reserve_id}"
        return requests.get(**req).json()


    def reserve(self):
        if self.done():
            return False
        
        if not self.reserve_id:
            success = self.check_booking()
            if not success:
                return

        perform = self.zone.perform

        req = baseData.copy()
        req['url'] = 'https://api.allticket.com/payment/payment/outlet'
        req['json'] = {
            "payment_channel": "outlet",
            "performId": perform.id,
            "reserveId": self.reserve_id,
            "insureProducts": []
        }

        if self.done():
            return False
        
        res = requests.post(**req).json()

        if res['success'] == True:
            perform.seat_needed -= 1
            return True


In [12]:
perf = PerformManager(token, event, 1)


In [25]:
async def test():
    print('OLO')
    await asyncio.sleep(3)
    return 10

li = []
for i in range(10):
    print('HI')
    li.append(asyncio.create_task(test()))

await asyncio.gather(*li)


HI
HI
HI
HI
HI
HI
HI
HI
HI
HI
OLO
OLO
OLO
OLO
OLO
OLO
OLO
OLO
OLO
OLO


[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]